# lecture2 - abstract program abstract case study
绝大多数的 MLC 过程都是张量程序的变换。主要有两个问题：
- 张量程序可能的抽象可能有哪些
- 有哪些可能的张量程序变换

张量程序的一种抽象：TensorIR
- TensorIR 是一种在 TVM 中的张量程序抽象

In [ ]:
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
import numpy as np
from utils import code2html

TenosrIR 最主要的作用为表示循环和各种硬件设备可能需要的各种专门的硬件指令和内存访问加速方法（e.g Threading）
Example: 使用两个维度为 128 $\times$ 128 的矩阵 A, B 进行如下计算(linear + relu):
- $Y = matmul(A, B): Y_{ij} = \sum_k A_{ik}B_{kj}$
- $C = relu(Y): C_{ij} = max(Y_{ij}, 0)$

使用 numpy 可以轻松实现

In [2]:
# numpy
import numpy as np
dtype = "float32"
mat_a_np = np.random.rand(128, 128).astype(dtype)
mat_b_np = np.random.rand(128, 128).astype(dtype)
# in numpy a @ b = np.matmul(a, b)
c_mm_relu = np.maximum(mat_a_np @ mat_b_np, 0)

## 1.low-level numpy
为了展示其中的细节，我们使用 low-level numpy 进行实现

In [3]:
def low_level_np_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype=dtype)
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)

验证 low level numpy 的结果

In [4]:
c_low_level_np_mm_relu = np.empty((128, 128), dtype=dtype)
low_level_np_mm_relu(mat_a_np, mat_b_np, c_low_level_np_mm_relu)
np.testing.assert_allclose(c_mm_relu, c_low_level_np_mm_relu, rtol=1e-6)

## 2.TVMScript

下面为 TVMScript 对 mm_relu 的一种实现

尽管运行速度比较慢但是仍然包含了在真实实现的过程中的可能的元素
- 多维数组
- 在多维度上的循环
- 在循环下执行的计算语句
下面使用 TensorIR 来实现 mm+relu,和上面 numpy 实现的 mm+relu 等价

In [5]:
@tvm.script.ir_module
class MyModuleWithAxisRemapSugar:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), dtype],
                B: T.Buffer[(128, 128), dtype],
                C: T.Buffer[(128, 128), dtype]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype=dtype)
        for i,j,k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

为什么需要额外的附加信息：
- 一个重要的观察结果是，附加信息（块轴范围及其属性）使块轴独立于外部循环嵌套 i, j, k。
- 块轴信息还提供了额外的属性，帮助我们验证用于执行计算的外部循环的正确性。例如，代码块会导致错误，如果我们只将它绑定到一个大小为 127 的 for 循环，因为循环需要一个大小为 128 的迭代器。
- 这些附加信息也有助于我们进行机器学习编译分析。例如，虽然我们总是可以在空间轴上做并行化，在规约轴上进行并行化将需要特定的策略。
```python
# SSR means the properties of each axes are "spatial", "spatial", "reduce"
vi, vj, vk = T.axis.remap("SSR", [i, j, k])
# 和这段程序等价

vi = T.axis.spatial(range_of_i, i)
vj = T.axis.spatial(range_of_j, j)
vk = T.axis.reduce(range_of_k, k)

In [6]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), dtype],
                B: T.Buffer[(128, 128), dtype],
                C: T.Buffer[(128, 128), dtype]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype=dtype)
        for i,j,k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

In [7]:
print(type(MyModule))
print(type(MyModule["mm_relu"]))

<class 'tvm.ir.module.IRModule'>
<class 'tvm.tir.function.PrimFunc'>


一个 IRModule 能够包含多个 PrimFunc, 下面为一个例子

In [8]:
@tvm.script.ir_module
class MyModuelWithMultiFunc:
    @T.prim_func
    def mm(A: T.Buffer[(128, 128), "float32"],
           B: T.Buffer[(128, 128), "float32"],
           Y: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "mm", "tir.noalias": True})
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
    
    @T.prim_func
    def relu(Y: T.Buffer[(128, 128), "float32"],
             C: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "relu", "tir.noalias": True})
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

## 3.张量程序变换
在上一节中，给出了如何使用低级 NumPy 编写 mm_relu 的示例。 在实践中，可以有多种方法来实现相同的功能，并且每种实现都可能导致不同的性能。

将讨论性能背后的原因以及如何在以后的内容中利用这些变体

下面为不同于上面的 `low_level_np_mm_relu` 的另一种一种low level numpy 的实现

In [9]:
def low_level_np_mm_relu_v2(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype=dtype)
    for i in range(128):
        for j0 in range(32):
            for k in range(128):
                for j1 in range(4):
                    j = j0 * 4 + j1
                    if k == 0:
                        Y[i, j] = 0
                    Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    for i in range(128):
        for j in range(128):
            C[i, j] = np.maximum(Y[i, j], 0)


上面的代码块显示了 mm_relu 的一个稍微不同的变体。它与原始程序的不同是 - 我们用两个循环 j0 和 j1 替换了 j 循环； - 迭代顺序略有变化。

为了获得 lnumpy_mm_relu_v2，我们必须重写一个新函数（或手动复制粘贴和编辑）。

TensorIR 引入了一个名为 Schedule 的辅助结构，它允许我们不必重新写代码就能做到这一点。

我们先查看变换前的 TVMScript 代码

In [10]:
print(MyModule.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j, k in tir.grid(128, 128, 128):
            with tir.block("Y"):
                vi, vj, vk = tir.axis.remap("SSR", [i, j, k])
                tir.reads(A[vi, vk], B[vk, vj])
                tir.writes(Y[vi, vj])
                with tir.init():
                    Y[vi, vj] = tir.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in tir.grid(128, 128):
            with tir.block("C"):
                vi, vj = tir.axis.remap("SS", [i, j])
                tir.reads(Y[vi, vj])
                tir.writes(C[vi, vj])
                C[vi, vj] = 

现在尝试进行变换得到 `low_level_np_mm_relu` 对应的程序

现在我们准备好尝试代码变换。我们首先创建一个以给定的 MyModule 作为输入的 Schedule 辅助类。

In [11]:
sch = tvm.tir.Schedule(MyModule)

然后执行下面的代码得到对应 Y 块的相应循环的引用

In [12]:
block_Y = sch.get_block("Y", func_name="mm_relu")
i, j, k = sch.get_loops(block_Y)

现在我们准备好进行变换了。我们将执行的第一个变换是将循环 j 分成两个循环，其中内部循环的长度为 4。请注意，变换是程序性的，因此如果你不小心执行了两次该代码块，我们将得到“变量 j 不再存在”的错误。如果发生这种情况，你可以从头（创建 sch 的位置）开始再次运行。

In [13]:
j0, j1 = sch.split(j, factors=[None, 4])

然后我们查看变换后的 TVMScript 代码, 使用 `sch.mod.script()`

In [14]:
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j_0, j_1, k in tir.grid(128, 32, 4, 128):
            with tir.block("Y"):
                vi = tir.axis.spatial(128, i)
                vj = tir.axis.spatial(128, j_0 * 4 + j_1)
                vk = tir.axis.reduce(128, k)
                tir.reads(A[vi, vk], B[vk, vj])
                tir.writes(Y[vi, vj])
                with tir.init():
                    Y[vi, vj] = tir.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in tir.grid(128, 128):
            with tir.block("C"):
                vi, vj = tir.axis.remap("SS", [i, j])
       

然后我们使用 reorder 交换 j1 和 k 的顺序

In [15]:
sch.reorder(j0, k, j1)
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j_0, k, j_1 in tir.grid(128, 32, 128, 4):
            with tir.block("Y"):
                vi = tir.axis.spatial(128, i)
                vj = tir.axis.spatial(128, j_0 * 4 + j_1)
                vk = tir.axis.reduce(128, k)
                tir.reads(A[vi, vk], B[vk, vj])
                tir.writes(Y[vi, vj])
                with tir.init():
                    Y[vi, vj] = tir.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in tir.grid(128, 128):
            with tir.block("C"):
                vi, vj = tir.axis.remap("SS", [i, j])
       

## 4.执行另一种变换
现在我们获得了原来程序的一个变体， 在本节中，我们将继续进行另外两步变换以得到另一个变体.

我们使用名为 reverse_compute_at 的原语将块 C 移动到 Y 的内循环里。(即两个循环融合到一起，减少了循环开销)

In [16]:
block_C = sch.get_block("C", func_name="mm_relu")
sch.reverse_compute_at(block_C, j0)
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j_0 in tir.grid(128, 32):
            for k, j_1 in tir.grid(128, 4):
                with tir.block("Y"):
                    vi = tir.axis.spatial(128, i)
                    vj = tir.axis.spatial(128, j_0 * 4 + j_1)
                    vk = tir.axis.reduce(128, k)
                    tir.reads(A[vi, vk], B[vk, vj])
                    tir.writes(Y[vi, vj])
                    with tir.init():
                        Y[vi, vj] = tir.float32(0)
                    Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
            for ax0 in tir.serial(4):
                with tir.block("

到目前为止，我们将归约初始化和更新放在一个块体中。这种组合形式为循环变换带来了便利（因为初始化和更新的外循环 i、j 通常需要彼此保持同步）。

在循环变换之后，我们可以将 Y 元素的初始化与归约更新分开。我们可以通过 decompose_reduction 原语来做到这一点。（注意：这也是 TVM 在以后编译的时候隐式做的，所以这一步的主要目的是让它显式，看看最终效果）。

In [17]:
sch.decompose_reduction(block_Y, k)
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i, j_0 in tir.grid(128, 32):
            for j_1_init in tir.serial(4):
                with tir.block("Y_init"):
                    vi = tir.axis.spatial(128, i)
                    vj = tir.axis.spatial(128, j_0 * 4 + j_1_init)
                    tir.reads()
                    tir.writes(Y[vi, vj])
                    Y[vi, vj] = tir.float32(0)
            for k, j_1 in tir.grid(128, 4):
                with tir.block("Y_update"):
                    vi = tir.axis.spatial(128, i)
                    vj = tir.axis.spatial(128, j_0 * 4 + j_1)
                    vk = tir.axis.red

最后得到的程序类似于下面的 low level numpy 程序

In [19]:
def low_level_np_mm_relu_v3(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    Y = np.empty((128, 128), dtype="float32")
    for i in range(128):
        for j0 in range(32):
            # Y_init
            for j1 in range(4):
                j = j0 * 4 + j1
                Y[i, j] = 0
            # Y_update
            for k in range(128):
                for j1 in range(4):
                    j = j0 * 4 + j1
                    Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
            # C
            for j1 in range(4):
                j = j0 * 4 + j1
                C[i, j] = max(Y[i, j], 0)

c_np = np.empty((128, 128), dtype=dtype)
low_level_np_mm_relu_v3(mat_a_np, mat_b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)

需要注意的是我们需要习惯增量代码变换的范例。很多时候我们不需要从头开始写一遍算法，只需要在原有代码的基础之上进行程序变换就行。

## 5.构建与运行
到目前为止，我们只查看了变换结果的 TVMScript 输出。我们也可以运行 IRModule 中得到的程序。

首先，我们调用构建函数将 IRModule 变换为 runtime.Module，它表示可运行函数的集合。 这里 target 指定了部署环境的详细信息。对于现在这种特殊情况，我们将使用 llvm，它可以帮助我们编译到本机 CPU 平台。

当我们针对不同的平台（例如 Android 手机）或具有特殊说明的平台（Intel Skylake）时，我们需要相应地调整 target。当我们开始部署到这些环境时，我们将讨论不同的目标选择。

In [23]:
rt_lib = tvm.build(MyModule, target="llvm")

准备数据

In [20]:
a_nd = tvm.nd.array(mat_a_np)
b_nd = tvm.nd.array(mat_b_np)
c_nd = tvm.nd.empty((128, 128), dtype="float32")
type(c_nd)

tvm.runtime.ndarray.NDArray

最后，我们可以从 rt_lib 中获取可运行函数，并通过传递三个数组参数来执行它。我们可以进一步运行验证来检查代码差异。

In [24]:
func_mm_relu = rt_lib["mm_relu"]
func_mm_relu(a_nd, b_nd, c_nd)

np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

我们已经构建并运行了原始的 MyModule。 我们还可以构建变换后的程序。

In [25]:
rt_lib_after = tvm.build(sch.mod, target="llvm")
rt_lib_after["mm_relu"](a_nd, b_nd, c_nd)
np.testing.assert_allclose(c_mm_relu, c_nd.numpy(), rtol=1e-5)

最后，我们可以比较一下两者的时间差。 time_evaluator 是一个辅助的测试函数，可用于比较不同生成函数的运行性能。

In [40]:
f_timer_before = rt_lib.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of MyModule %g sec" % f_timer_before(a_nd, b_nd, c_nd).mean)
f_timer_after = rt_lib_after.time_evaluator("mm_relu", tvm.cpu())
print("Time cost of transformed sch.mod %g sec" % f_timer_after(a_nd, b_nd, c_nd).mean)

Time cost of MyModule 0.00287374 sec
Time cost of transformed sch.mod 0.000602236 sec


## 5.尝试使用不同的 j_factor
尝试使用不同的 $j_{factor}$ ，然后通过 `time_evaluator` 进行评估

In [51]:
# 使用 tansform 函数计算出对应的 j_factor 的 TVMScript
def transform(mod: tvm.IRModule, j_factor: int):
    sch = tvm.tir.Schedule(mod)
    block_Y = sch.get_block("Y", func_name="mm_relu")
    i, j, k = sch.get_loops(block_Y)
    j0, j1 = sch.split(j, factors=[None, j_factor])
    sch.reorder(j0, k, j1)
    block_C = sch.get_block("C", func_name = "mm_relu")
    sch.reverse_compute_at(block_C, j0)
    return sch.mod

for i in range(8):
    j_factor = 2**i
    mod_transformed = transform(MyModule, j_factor=j_factor)
    transformed_rt_lib = tvm.build(mod_transformed, target="llvm")
    f_timer_before = transformed_rt_lib.time_evaluator("mm_relu", tvm.cpu())
    time = f_timer_before(a_nd, b_nd, c_nd).mean
    print(f"Time cost of MyModule {time:.7f} sec, j factor is {j_factor}")

Time cost of MyModule 0.0028722 sec, j factor is 1
Time cost of MyModule 0.0026224 sec, j factor is 2
Time cost of MyModule 0.0012988 sec, j factor is 4
Time cost of MyModule 0.0005892 sec, j factor is 8
Time cost of MyModule 0.0010734 sec, j factor is 16
Time cost of MyModule 0.0004859 sec, j factor is 32
Time cost of MyModule 0.0003095 sec, j factor is 64
Time cost of MyModule 0.0002956 sec, j factor is 128


## 6.使用张量表达式生成 TensorIR 代码
在许多情况下，我们的开发形式是不在循环级别的更高级别的抽象。 所以另一种常见的获取 TensorIR 的方式是务实地生成相关代码。

张量表达式 (TE) 是一种特定领域的语言，它通过 API 之类的表达式描述一系列计算。

In [52]:
from tvm import te

A = te.placeholder((128, 128), "float32", name="A")
B = te.placeholder((128, 128), "float32", name="B")
k = te.reduce_axis((0, 128), "k")
Y = te.compute((128, 128), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="Y")
C = te.compute((128, 128), lambda i, j: te.max(Y[i, j], 0), name="C")


In [54]:
te_func = te.create_prim_func([A, B, C]).with_attr({"global_symbol": "mm_relu"})
MyModuleFromTE = tvm.IRModule({"mm_relu": te_func})
print(MyModuleFromTE.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(128, 128), "float32"], B: tir.Buffer[(128, 128), "float32"], C: tir.Buffer[(128, 128), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # body
        # with tir.block("root")
        Y = tir.alloc_buffer([128, 128], dtype="float32")
        for i0, i1, i2 in tir.grid(128, 128, 128):
            with tir.block("Y"):
                i, j, k = tir.axis.remap("SSR", [i0, i1, i2])
                tir.reads(A[i, k], B[k, j])
                tir.writes(Y[i, j])
                with tir.init():
                    Y[i, j] = tir.float32(0)
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
        for i0, i1 in tir.grid(128, 128):
            with tir.block("C"):
                i, j = tir.axis.remap("SS", [i0, i1])
                tir.reads(Y[i, j])
                tir.writes(C[i, j])
                C[i, j] = tir.max(Y[i, j], 